Dieser Quellcode ist Bestandteil der Dissertation von Ines Reinecke
vorgelegt am 11.07.2023 an der Technischen Universität Dresden, Medizinische Fakultät

Dieses Script enthält:
* Einlesen der Daten DS-Med als Eingangsgröße (scatter_input)
* Einlesen der ATC Codes Version 2022 - mit den entsprechenden ATC Beschreibungen in Deutsch
* Zusammenführen der Medikationsverordnungen mit den ATC Codes und den Beschreibungen
* Generieren eines Streudiagramms mit der Bibliothek Bokeh, interaktiv

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, CustomJS, HoverTool, NumeralTickFormatter, CellFormatter, Legend
from bokeh.models.widgets import TextInput, Button, TableColumn,DataTable, Div
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file, export_svg
from bokeh.events import ButtonClick
from bokeh.models import HoverTool, PrintfTickFormatter

In [8]:
# Lesen der Medikationsverordnungen und ATC und Beschreibungen in Deutsch
    
df=pd.read_csv('../data_in/scatter_input.csv')
df_atc=pd.read_csv('../data_in/atc_codes2022.csv', sep=";", encoding="ISO-8859-1")


# Reduzieren des Dataframes auf die ATC Codes und den Anteil der strukturierten und unstrukturiert vorliegenden Daten in den Medikationsverordnungen, sowie die Gesamtmenge
df=df[['ATC_CORRECT','unstructured','structured','total']]

# Reduzieren des Dataframes mit den ATC Codes
df_atc=df_atc[['ATC-Code','ATC-Bedeutung','DDD-Info']]
# Umbenennen bestimmter Spalten des Dataframes mit den ATC Codes
df_atc.columns=['ATC_CORRECT','agent','Daily drug dosage']

# Zusammenführen (Merge) der Dataframes, sodass die auch die Beschreibungen für die ATC Codes in Deutsch in dem Streudiagramm verfügbar und auch als Wirkstoffe suchbar sind
df=pd.merge(df,df_atc, how='left', on='ATC_CORRECT')


In [10]:

# Streudiagramm
  
df['prop_struct']= round(df.structured / df.total, 2)
df['prop_unstruct']= round(df.unstructured  / df.total, 2)
df['prop_unstruct_hover']= round((df.unstructured  / df.total)*100, 1)
mean_mapped=df.unstructured.sum() / df.total.sum()
std_mean=df.prop_unstruct.std() / np.sqrt(len(df.prop_unstruct))
df['rel_unstruct_mean']=df.prop_unstruct / mean_mapped #calc relation of unstructured data to mean for atc-code

#filter all ATC codes with 0 counts:
df=df[df.total>=1]
df.sort_values('total', axis=0, ascending=False, inplace=True)
    
df.loc[df["prop_unstruct"]>0.50, 'color'] = "#a3001e"
df.loc[df["prop_unstruct"]<= 0.50, 'color'] = "#2c35b3"


#Neuen Bokeh Plot bauen: 
plot_source = ColumnDataSource(df)

sourcetext = ColumnDataSource(data={'ATC_CORRECT': [], 'agent': [], 'total': [], "prop_unstruct": [], "color": []})
    
#Callback Funktion, um alle "markierten" Punkte wieder aufzuheben
clearbutton = Button(label="Filter löschen", button_type="primary", width=50)
    
clearbuttoncallback = CustomJS(args=dict(sourcetext=sourcetext), code="""
    var d2 = sourcetext.data;
           d2['ATC_CORRECT']=[]
           d2['agent']=[]
           d2['total']=[]
           d2['prop_unstruct']=[]
           d2['color']=[]
    sourcetext.change.emit();""")
clearbutton.js_on_event(ButtonClick, clearbuttoncallback)
    

# Suchfeld erstellen
search = TextInput(value = "", placeholder = "A1...", width=200)

# Definition einer Callback Funktion um die den Filter mit der Suche als String zu aktualisieren
callback_search = CustomJS(args=dict(source=plot_source, sourcetext=sourcetext), code="""
    var label = cb_obj.value.toLowerCase();
    var d1 = source.data;
    var d2 = sourcetext.data;
        
    for (var i = 0; i < d1['ATC_CORRECT'].length; i++) {
        if (d1['agent'][i].toLowerCase().includes(label)|d1['ATC_CORRECT'][i].toLowerCase().includes(label)) { 
            d2['ATC_CORRECT'].push(d1['ATC_CORRECT'][i])
            d2['agent'].push(d1['agent'][i])
            d2['total'].push(d1['total'][i])
            d2['prop_unstruct'].push(d1['prop_unstruct'][i])
            d2['color'].push(d1['color'][i])
    }
        
}
sourcetext.data = d2;
sourcetext.change.emit();
""")

# Hinzufügen des Callbacks zu dem Eingabefeld für die Suche
search.js_on_change('value', callback_search) 
    

# Hover Funktion mit den Metainformationen definieren und hinzufügen
hover=HoverTool(tooltips=[("ATC Code", "@ATC_CORRECT"), ("Anzahl", "@total"), ("Anteil unstrukturiert", "@prop_unstruct_hover{0.00}%"),('Wirkstoff', '@agent')],formatters={"prop_unstruct_hover": "printf"})
    
# Plot des Streudiagramms
plot = figure(plot_width=1000, plot_height=900, x_axis_type="log",
                tools=[hover, 'pan', 'tap', 'lasso_select', 'wheel_zoom','box_zoom', 'reset','save'],
                title=None, toolbar_location="right", x_axis_label='Summe der Medikationsverordnungen (logarithmische Skala)',  y_axis_label='Anteil unstrukturierter Daten', x_range=(10,df.total.max()+10000))
plot.add_layout(Legend(), 'above')
plot.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
plot.yaxis.axis_label_text_font_size = "12pt"
plot.xaxis.axis_label_text_font_size = "12pt"
plot.xaxis.major_label_text_font_size = "12pt"
plot.yaxis.major_label_text_font_size = "12pt"
r1 = plot.circle('total', 'prop_unstruct', size=6, alpha=0.5, source=plot_source, name="plot",
            legend_label="ATC code (rot = überwiegend unstrukturiert, blau = überwiegend strukturiert)", color="color")
r2 = plot.circle('total', 'prop_unstruct', size=12, alpha=0.8, source=sourcetext, name="textplot", color="color")
r3 = plot.line([0.00001,1000000000], [mean_mapped, mean_mapped], legend_label="Durchschnitt unstrukturierter Daten: {:0.2%}".format(mean_mapped), 
            line_dash="dashed", line_color='grey', line_width=2)
plot.legend.label_text_font_size = "12pt"
hover.renderers =[r1]
   
#Hinzufügen der Tabelle mit den gefilterten Ergebnissen
    
columns = [TableColumn(field="ATC_CORRECT", title="ATC code"),TableColumn(field="agent", title="Wirkstoff"),
        TableColumn(field="total", title="Anzahl gesamt"), TableColumn(field="prop_unstruct", title="Anteil unstrukturierter Daten")]
div_table = Div(text="<br>   <br>", style={'font-size': '100%', 'color': 'black'})
table = DataTable(source=sourcetext, columns=columns, width=700, height=200, editable=False)
   
div_search = Div(text="<br>Eingabe ATC Code oder Wirkstoff:", style={'font-size': '100%', 'color': 'black'})
    
output_file('../data_results/03_data_transparency_results/streu_atc_struktur.html')
   
#layout:
search_col = column(div_search, search)
find_tools = column(search_col, clearbutton,)
table_col = column(div_table, table)

l=layout([
    #[div_header],
    [find_tools, table_col],
    [plot],
    [Div(text="<br><br><br>")]])
show(l)
    

In [ ]:
#pvalue difference overview
df_calculated=df_calculated[df_calculated.total>=10]
df_calculated.sort_values('ft_pvalue', axis=0, ascending=False, inplace=True)
df_calculated.head()

,ATC_CORRECT,unstructured,structured,total,agent,Daily drug dosage,prop_struct,prop_unstruct,rel_unstruct_mean,tt_pvalue,color_ttest,ft_pvalue,rel_ftest_unstruct,color
695,N05AX08,2282,3907,6189,Risperidon,"5 mg O; 2,7 mg P Depot",0.631281,0.368719,0.999190,0.961140,grey,0.968409,0.999186,grey
463,R03AC13,157,266,423,Formoterol,NaN,0.628842,0.371158,1.005801,0.927425,grey,0.919825,1.005803,grey
679,N05BA01,1779,3011,4790,Diazepam,"10 mg O,P,R",0.628601,0.371399,1.006453,0.732748,grey,0.730148,1.006476,grey
471,R06AX26,158,291,449,Fexofenadin,"0,12 g O",0.648107,0.351893,0.953594,0.452958,grey,0.463559,0.953580,grey
502,N06AA05,227,362,589,Opipramol,"0,15 g O",0.614601,0.385399,1.044392,0.410725,grey,0.417248,1.044412,grey
